In [26]:
# Snowpark for Python
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.types import StructType, StructField, DoubleType, StringType
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import col
from snowflake.ml.modeling.linear_model import LinearRegression
from snowflake.ml.modeling.metrics import mean_squared_error
from snowflake.snowpark.functions import call_builtin
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

# data science libs
import numpy as np

# misc
import json
import joblib

from sklearn.model_selection import train_test_split

In [3]:
# Make a Snowpark Connection
# Edit the connection.json before creating the session object below
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))



Connection Established with the following parameters:
User                        : GJWONG
Role                        : "ACCOUNTADMIN"
Database                    : "ML_CLI_DB"
Schema                      : "ML_CLI_SCHEMA"
Warehouse                   : "ML_CLI_WH"
Snowflake version           : 7.41.0
Snowpark for Python version : 1.5.1


In [37]:
# Your code goes here, inside the "main" handler.
tableName = 'weekly_weather_data'
# dataframe = session.table(tableName).filter(col("language") == 'python')
df = session.table(tableName)

# Split train and test sets
# df = df.to_pandas()
df.show()


----------------------------------------------------------------------------------------------------------------------------------------------------
|"WEEK_START_DATE"    |"AVG_CLDCVR"  |"AVG_FEELSLIKE"  |"AVG_PRECIP"  |"AVG_RELHUM"  |"AVG_SNOWFALL"  |"AVG_TEMP"  |"AVG_WINDDIR"  |"AVG_WINDSPD"  |
----------------------------------------------------------------------------------------------------------------------------------------------------
|2009-12-28 00:00:00  |75.025000     |36.250000        |0.000000      |88.725000     |0.000000        |39.462500   |170.350000     |5.700000       |
|2010-01-04 00:00:00  |72.559524     |36.184524        |0.000000      |89.083333     |0.000000        |39.333333   |170.750000     |5.666667       |
|2010-01-11 00:00:00  |70.875000     |36.886905        |0.000000      |89.672619     |0.000000        |39.839286   |168.755952     |5.386905       |
|2010-01-18 00:00:00  |68.773810     |37.440476        |0.000000      |89.107143     |0.000000        |40.

In [6]:
df.columns

Index(['MONTH_START_DATE', 'AVG_CLDCVR', 'AVG_FEELSLIKE', 'AVG_PRECIP',
       'AVG_RELHUM', 'AVG_SNOWFALL', 'AVG_TEMP', 'AVG_WINDDIR', 'AVG_WINDSPD'],
      dtype='object')

In [40]:
session.close()

In [38]:
# X = df[['AVG_CLDCVR', 'AVG_RELHUM', 'AVG_TEMP', 'AVG_WINDSPD']]
# y = df['AVG_FEELSLIKE']
# features = ['AVG_CLDCVR', 'AVG_RELHUM', 'AVG_TEMP', 'AVG_WINDSPD']
# features = ['AVG_CLDCVR', 'AVG_RELHUM', 'AVG_WINDSPD']
features = ['AVG_PRECIP']
# label = ['AVG_FEELSLIKE']
label = ['AVG_TEMP']
# pred = ['PREDICTED_FEELSLIKE']
pred = ['PREDICTED_TEMP']

train_df, test_df = df.random_split(weights=[0.8, 0.2], seed=0)

regressor = XGBRegressor(
    input_cols=features,
    label_cols=label,
    output_cols=pred
)

# Train
regressor.fit(train_df)

# Predict
result = regressor.predict(test_df)

# Train the linear regression model
# reg = LinearRegression().fit(X_train, y_train)
# reg = call_builtin("linear_regressor", df, features, label)
# print(reg.score(X_train, y_train))
# y_pred = reg.predict(X_test)
# print(mean_squared_error(y_test, y_pred))


[15:32:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



/usr/local/anaconda3/lib/python3.9/site-packages/snowflake/ml/model/_signatures/core.py:153: UserWarning: Warning: Type DecimalType(38, 6) is being automatically converted to DOUBLE in the Snowpark DataFrame. This automatic conversion may lead to potential precision loss and rounding errors. If you wish to prevent this conversion, you should manually perform the necessary data type conversion.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/snowflake/ml/_internal/utils/snowpark_dataframe_utils.py:88: UserWarning: Warning: The Decimal(38, 6) data type is being automatically converted to DoubleType in the Snowpark DataFrame. This automatic conversion may lead to potential precision loss and rounding errors. If you wish to prevent this conversion, you should manually perform the necessary data type conversion.
  warnings.warn(
The version of package numpy in the local environment is 1.26.1, which does not fit the criteria for the requirement numpy==1.26.0. Your UDF might

In [39]:
# mape = mean_absolute_percentage_error(df=result, 
#                                         y_true_col_names="AVG_FEELSLIKE", 
#                                         y_pred_col_names="PREDICTED_FEELSLIKE")

# result.select("AVG_FEELSLIKE", "PREDICTED_FEELSLIKE").show()
# print(f"Mean absolute percentage error: {mape}")


mape = mean_absolute_percentage_error(df=result, 
                                        y_true_col_names="AVG_TEMP", 
                                        y_pred_col_names="PREDICTED_TEMP")
mse = mean_squared_error(df=result, 
                        y_true_col_names="AVG_TEMP", 
                        y_pred_col_names="PREDICTED_TEMP")

# r2 = r2_score(df=result, 
#                 y_true_col_names="AVG_TEMP", 
#                 y_pred_col_names="PREDICTED_TEMP")
result.select("AVG_TEMP", "PREDICTED_TEMP").show()
print(f"Mean absolute percentage error: {mape}")
print(f"Mean squared error: {mse}")
# print(F"R2 score: {r2}")

----------------------------------
|"AVG_TEMP"  |"PREDICTED_TEMP"   |
----------------------------------
|42.803571   |50.91461563110352  |
|44.285714   |50.91461563110352  |
|46.577381   |50.91461563110352  |
|49.583333   |50.91461563110352  |
|54.333333   |50.91461563110352  |
|57.690476   |50.91461563110352  |
|58.220238   |50.91461563110352  |
|62.744048   |50.91461563110352  |
|64.095238   |50.91461563110352  |
|41.833333   |50.91461563110352  |
----------------------------------

Mean absolute percentage error: 0.1469382340679494
Mean squared error: 77.74850222239256


In [12]:
y

0      37.000000
1      37.106183
2      39.805060
3      41.875000
4      46.738889
         ...    
152    64.278226
153    59.161111
154    50.728495
155    42.329167
156    36.771084
Name: AVG_FEELSLIKE, Length: 157, dtype: float64